In [ ]:
%load_ext autoreload
%autoreload 2

import os,sys
import h5py  
import numpy as np
import math,cmath
from scipy.optimize import curve_fit
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams.update({"axes.grid" : True})

if not os.path.abspath('../util') in sys.path:
    sys.path.append(os.path.abspath('../util'))
import util as yu

lat=yu.LatticeEnsemble('cA2.09.48')

In [ ]:
(lat.getaEN(0)+lat.getaEpi(0))*lat.aInv

In [ ]:
(lat.getaEN(4)+lat.getaEpi(4))*lat.aInv

In [ ]:
# load data

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA2.09.48/NJNpi_N0pi+/proj/'
with h5py.File(basePath+'N_2pt.h5') as f:
    cfgList=list(yu.deepKey(f,3).keys())

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA2.09.48/NJN/proj/'
with h5py.File(basePath+'protonup10.h5') as f:
    cfgList=list(set(cfgList)&set(yu.deepKey(f,3).keys()))
with h5py.File(basePath+'protonup12.h5') as f:
    cfgList=list(set(cfgList)&set(yu.deepKey(f,3).keys()))

print(len(cfgList))

basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA2.09.48/NJNpi_N0pi+/proj/'

filesDic={}
files2ptKey=['NN','NNpi','NpiNpi']
filesDic['NN']=['N_2pt']
filesDic['NNpi']=['T']
filesDic['NpiNpi']=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
files3ptKey=['NJNpi']
filesDic['NJNpi']=['B','W','Z']

f={}
for key in filesDic.keys():
    f[key]={}
    for file in filesDic[key]:
        f[key][file] = h5py.File(basePath + file + '.h5','r')

data={}
for key in files2ptKey:
    data[key]={}
    for file in filesDic[key]:
        data[key][file]={}
        tF=f[key][file]
        for projaStr in tF.keys():
            proja=eval(projaStr)
            data[key][file][proja]={}
            for projbStr in tF[projaStr].keys():
                projb=eval(projbStr)
                data[key][file][proja][projb]={}
                for proj2 in tF[projaStr][projbStr].keys():
                    data[key][file][proja][projb][proj2]={}
                    for cont in tF[projaStr][projbStr][proj2][cfgList[0]].keys():
                        data[key][file][proja][projb][proj2][cont]=np.array([
                            f[key][file][projaStr][projbStr][proj2][cfg][cont][()] for cfg in cfgList
                            ])

for key in files3ptKey:
    data[key]={}
    for file in filesDic[key]:
        data[key][file]={}
        tF=f[key][file]
        for projaStr in tF.keys():
            proja=eval(projaStr)
            data[key][file][proja]={}
            for projbStr in tF[projaStr].keys():
                projb=eval(projbStr)
                data[key][file][proja][projb]={}
                for proj2 in tF[projaStr][projbStr].keys():
                    data[key][file][proja][projb][proj2]={}
                    for cont in tF[projaStr][projbStr][proj2][cfgList[0]].keys():
                        data[key][file][proja][projb][proj2][cont]=np.array([
                            f[key][file][projaStr][projbStr][proj2][cfg][cont][()] for cfg in cfgList
                            ])

for key in filesDic.keys():
    for file in filesDic[key]:
        f[key][file].close()


basePath='/p/project/pines/li47/code/projectData/nucleon_sigma_term/cA2.09.48/NJN/proj/'
filesDic={}
files3ptKey=['NJN']
filesDic['NJN']=['protonup10','protondn10','protonup12','protondn12']

f={}
for key in filesDic.keys():
    f[key]={}
    for file in filesDic[key]:
        f[key][file] = h5py.File(basePath + file + '.h5','r')

for key in files3ptKey:
    data[key]={}
    for file in filesDic[key]:
        data[key][file]={}
        tF=f[key][file]
        for projaStr in tF.keys():
            proja=eval(projaStr)
            data[key][file][proja]={}
            for projbStr in tF[projaStr].keys():
                projb=eval(projbStr)
                data[key][file][proja][projb]={}
                for proj2 in tF[projaStr][projbStr].keys():
                    data[key][file][proja][projb][proj2]={}
                    for cont in tF[projaStr][projbStr][proj2][cfgList[0]].keys():
                        data[key][file][proja][projb][proj2][cont]=np.array([
                            f[key][file][projaStr][projbStr][proj2][cfg][cont][()] for cfg in cfgList
                            ])

for key in filesDic.keys():
    for file in filesDic[key]:
        f[key][file].close()


In [ ]:
mom2Dic={}
mom2Dic[(0,0,0)]='N1\\pi1'
mom2Dic[(0,0,1)]=mom2Dic[(0,0,-1)]=mom2Dic[(1,0,0)]='N1\\pi0'

def getDat2pt(p_tot,a,b):
    if a==b==0:
        proj=(p_tot,'a')
        return data['NN']['N_2pt'][proj][proj]['P0']['NP']
    elif a==0:
        proja=(p_tot,'a')
        projb=(p_tot,(mom2Dic[p_tot],'a'))
        return 1j*np.sum([data['NNpi']['T'][proja][projb]['P0'][cont] for cont in ['Tseq'+str(i) for i in range(11,15)]],axis=0)
    elif b==0:
        return np.conj(getDat2pt(p_tot,b,a))
    else:
        proja=(p_tot,(mom2Dic[p_tot],'a'))
        projb=(p_tot,(mom2Dic[p_tot],'a'))
        diagList=['B_2pt','W_2pt','Z_2pt','M_correct_2pt']
        contDic={'B_2pt':['B'+str(i) for i in range(13,17)],'W_2pt':['W'+str(i) for i in range(25,29)],'Z_2pt':['Z'+str(i) for i in range(5,7)],'M_correct_2pt':['MN0PP']}
        return np.sum([
        np.sum([data['NpiNpi'][diag][proja][projb]['P0'][cont] for cont in contDic[diag]],axis=0)
        for diag in diagList],axis=0)

def getDat2ptMat(p_tot,N_op):
    t=np.transpose([[getDat2pt(p_tot,a,b) for b in range(N_op)] for a in range(N_op)],[2,3,0,1])
    t=(t+np.conj(np.transpose(t,[0,1,3,2])))/2
    return t

def getDat3pt(Gc,iso,proj,pi,tf,a,b):
    i_Gc={'id':0,'gx':1,'gy':2,'gz':3,'gt':4,'g5':5,'g5gx':6,'g5gy':7,'g5gz':8,'g5gt':9}[Gc]
    sgn_iso={'+':+1,'-':-1}[iso]
    if a==b==0:
        proja=((0,0,0),'a'); projb=(pi,'a')
        return data['NJN']['protonup'+str(tf)][proja][projb][proj]['MprotonUp'][:,1:tf,i_Gc]+sgn_iso*data['NJN']['protondn'+str(tf)][proja][projb][proj]['MprotonDn'][:,1:tf,i_Gc]
    elif a==0:
        proja=((0,0,0),'a'); projb=(pi,(mom2Dic[pi],'a'))
        contDic = {'B': ['B'+str(i) for i in range(9, 13)], 'Wu': ['W'+str(i) for i in range(17, 21)],
            'Wd': ['W'+str(i) for i in range(21, 25)], 'Z': ['Z'+str(i) for i in range(11, 15)]}
        return np.sum([np.sum([data['NJNpi'][diag[0]][proja][projb][proj][cont+'_deltat_'+str(tf)][:,1:tf,i_Gc] for cont in contDic[diag]],axis=0)
         for diag in ['B','Wu']],axis=0) + sgn_iso * np.sum([np.sum([data['NJNpi'][diag[0]][proja][projb][proj][cont+'_deltat_'+str(tf)][:,1:tf,i_Gc] for cont in contDic[diag]],axis=0)
         for diag in ['Wd','Z']],axis=0)
    elif b==0:
        return np.conj(getDat3pt(Gc,iso,proj,pi,tf,b,a)[:,::-1])
    else:
        return np.zeros([len(cfgList),tf-1])

def getDat3ptMat(Gc,iso,proj,pi,tf,N_op):
    t=np.transpose([[getDat3pt(Gc,iso,proj,pi,tf,a,b) for b in range(N_op)] for a in range(N_op)],[2,3,0,1])
    return t

tfList=[10,12]
def getDat(Gc,iso,proj,pi,N_op):
    proj2=proj
    if pi==(0,0,0) and proj=='Pzt':
        proj2='Pz'
    dat2pta=getDat2ptMat((0,0,0),N_op)
    dat2ptb=getDat2ptMat(pi,N_op)
    dat3pt={}
    for tf in tfList:
        dat3pt[tf]=getDat3ptMat(Gc,iso,proj2,pi,tf,N_op)
    t=np.array([
        np.array([dat2pta[cfg],dat2ptb[cfg]]+[dat3pt[tf][cfg] for tf in tfList],dtype=object)
    for cfg in range(len(cfgList))], dtype=object)
    return t


In [ ]:
(row, col) = (2, 4)
fig, axs = plt.subplots(row, col, figsize=(5*col, 4*row), squeeze=False)

tRef=1
N_op=2

for i_ptot in range(4):
    p_tot=[(0,0,0),(0,0,1),(0,0,-1),(1,0,0)][i_ptot]
    dat=getDat2ptMat(p_tot,N_op)

    def tFunc(dat):
        t=np.mean(dat,axis=0)
        (eVals,eVecs)=yu.GEVP(t,tRef)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))
        return En.T

    (mean,cov,err)=yu.jackknife(dat,tFunc)
    tmin=1;tmax=15

    for i in range(2):
        ttmax=tmax
        tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
        axs[0,i_ptot].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')

    axs[0,i_ptot].set_title('p_tot='+str(p_tot))
    if i_ptot==0:
        axs[0,i_ptot].fill_between([tmin,tmax],lat.amN-0.0011,lat.amN+0.0011,color='cyan',alpha=1)
        axs[0,i_ptot].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
    if i_ptot !=0:
        axs[0,i_ptot].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
        axs[0,i_ptot].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan')
        axs[0,i_ptot].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='r')
        axs[0,i_ptot].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='g')
        axs[0,i_ptot].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='b')

    axs[0,i_ptot].set_xticks(range(0,tmax,2))
    axs[0,i_ptot].set_ylim(.4,1.2)
    None

for i_ptot in range(4):
    p_tot=[(0,0,0),(0,0,1),(0,0,-1),(1,0,0)][i_ptot]
    dat=getDat2ptMat(p_tot,N_op)

    def tFunc(dat):
        t=np.mean(dat,axis=0)
        (eVals,eVecs)=yu.GEVP(t,tRef)
        t=eVecs[:,:,0].T
        return np.abs(t)

    (mean,cov,err)=yu.jackknife(dat,tFunc)
    tmin=1;tmax=15

    print(mean[0,10]/mean[1,10])

    for i in range(2):
        ttmax=tmax
        tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
        axs[1,i_ptot].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')

    axs[1,i_ptot].set_xticks(range(0,tmax,2))
    axs[1,i_ptot].set_yscale('log')
    axs[1,i_ptot].set_ylim(1e-1,1e4)
    axs[1,i_ptot].set_yticks([10,100,1000,10000])
    None

In [ ]:
# for report

(row, col) = (2, 4)
fig, axs = plt.subplots(row, col, figsize=(5*col, 4*row), squeeze=False)

tRef=1
N_op=2

for i_ptot in range(2):
    p_tot=[(0,0,0),(0,0,1),(0,0,-1),(1,0,0)][i_ptot]
    dat=getDat2ptMat(p_tot,N_op)

    def tFunc(dat):
        t=np.mean(dat,axis=0)
        (eVals,eVecs)=yu.GEVP(t,tRef)
        En=np.log(eVals/np.roll(eVals,-1,axis=0))
        return En.T

    (mean,cov,err)=yu.jackknife(dat,tFunc)
    tmin=1;tmax=15

    for i in range(2):
        ttmax=tmax
        tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
        axs[0,i_ptot].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')

    axs[0,i_ptot].set_title('p_tot='+str(p_tot))
    if i_ptot==0:
        axs[0,i_ptot].fill_between([tmin,tmax],lat.amN-0.0011,lat.amN+0.0011,color='cyan',alpha=1)
        axs[0,i_ptot].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
    if i_ptot !=0:
        axs[0,i_ptot].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
        axs[0,i_ptot].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan')

    axs[0,i_ptot].set_xticks(range(0,tmax,2))
    axs[0,i_ptot].set_ylim(.4,1.2)
    None

for i_ptot in range(4):
    p_tot=[(0,0,0),(0,0,1),(0,0,-1),(1,0,0)][i_ptot]
    dat=getDat2ptMat(p_tot,N_op)

    def tFunc(dat):
        t=np.mean(dat,axis=0)
        (eVals,eVecs)=yu.GEVP(t,tRef)
        t=eVecs[:,:,0].T
        return np.abs(t)

    (mean,cov,err)=yu.jackknife(dat,tFunc)
    tmin=1;tmax=15

    print(mean[0,10]/mean[1,10])

    for i in range(2):
        ttmax=tmax
        tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
        axs[1,i_ptot].errorbar(np.arange(tmin,ttmax),tMean,tErr,fmt='s',elinewidth=2,capsize=6,mfc='white')

    axs[1,i_ptot].set_xticks(range(0,tmax,2))
    axs[1,i_ptot].set_yscale('log')
    axs[1,i_ptot].set_ylim(1e-1,1e4)
    axs[1,i_ptot].set_yticks([10,100,1000,10000])
    None

In [ ]:
# for report

def run2ptGEVP(ops,title=None,ylim1=[.4,1.1],ylim2=[1e-4,1.1],axs=None,isospinCheck=False,xshift=0,projgs=['P01']):
    N_op=len(ops); tRef=4
    tmin=1; tmax=15

    if axs is None:
        fig, axs = yu.getFigAxs(1,2,Lrow=4*2,Lcol=5*2)
        title=', '.join([op[0] for op in ops]) + (': '+title if title!=None else '')
        # fig.suptitle(title)

    yu.addColHeader(axs,['Effective mass from E-values','Ratio from E-vectors'])
    colorDic={0:'r',1:'g',2:'b',3:'purple',4:'orange',5:'cyan',6:'pink'}
    fmtDic={0:'s',1:'d',2:'o'}

    for projg in projgs:
        xshift_internal=0 if len(projgs)==1 else {'P0':0,'P01':-0.3,'P02':0.3}[projg]
        xshift_internal+=xshift
        dat=getDat2ptMat((0,0,0),2)

        irow=0; icol=0
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eVals,eVecs)=yu.GEVP(t,tRef)
            En=np.log(eVals/np.roll(eVals,-1,axis=0))
            return En.T
        
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        for i in range(N_op):
            ttmax=tmax
            tMean=mean[i,tmin:ttmax];tErr=err[i,tmin:ttmax]
            print(['red','green'][i])
            for j in range(len(tMean)):
                print(tmin+j,tMean[j],tErr[j])

            axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt='s',color=colorDic[i],elinewidth=2,capsize=6,mfc='white')
        axs[irow,icol].set_xticks(range(0,tmax,1))
        axs[irow,icol].set_ylim(ylim1)

        if ops[0][1][0] == (0,0,0):
            axs[irow,icol].hlines(y=lat.getaEN(0),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan')
        elif ops[0][1][0] == (0,0,1):
            axs[irow,icol].hlines(y=lat.getaEN(1),xmin=tmin,xmax=tmax,color='cyan')
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(0),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(0)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='cyan') 
            axs[irow,icol].hlines(y=lat.getaEN(2)+lat.getaEpi(1),xmin=tmin,xmax=tmax,color='orange') 
            axs[irow,icol].hlines(y=lat.getaEN(1)+lat.getaEpi(2),xmin=tmin,xmax=tmax,color='orange') 

        icol=1
        for j in range(1):
            if isospinCheck and j!=0:
                continue
            def tFunc(dat):
                t=np.mean(dat,axis=0)
                (eVals,eVecs)=yu.GEVP(t,tRef)
                t=eVecs[:,:,j].T
                t=t/t[0,:]
                return np.abs(t)
                return np.array([np.real(t[0]),np.imag(t[1]),np.imag(t[2])])
            (mean,cov,err)=yu.jackknife(dat,tFunc)
            mean0=mean[j,tmin:tmax]
        
            for i in range(2):
                ttmax=tmax
                tMean=mean[i,tmin:ttmax]/mean0;tErr=err[i,tmin:ttmax]/mean0

                if isospinCheck:
                    t={'p':1,'n&pi+':np.sqrt(2/3),'p&pi0':np.sqrt(1/3)}[ops[i][0]]
                    tMean/=t; tErr/=t
                    
                # temp_min=8
                # t_factor=1 if i!=1 else 1/np.sqrt(2)
                # print(i)
                # print(yu.npRound(tMean[temp_min:]*t_factor,3))
                # print(yu.npRound(tErr[temp_min:]*t_factor,3))

                print(['square','circle'][i])
                for t_j in range(len(tMean)):
                    print(tmin+t_j,tMean[t_j],tErr[t_j])

                axs[irow,icol].errorbar(np.arange(tmin,ttmax)+xshift_internal,tMean,tErr,fmt=fmtDic[i],color=colorDic[j],elinewidth=2,capsize=6,mfc='white')
            
            axs[irow,icol].set_xticks(range(0,tmax,1))
            axs[irow,icol].set_ylim(ylim2)
            axs[irow,icol].set_yscale('log')
            axs[irow,icol].set_yticks([1,1e-1,1e-2,1e-3])

pt=(0,0,0); op1=('p',(pt,'a')); op2=('12',(pt,('N1\\pi1','a')))

ops=[op1,op2]
run2ptGEVP(ops)

In [ ]:
(row, col) = (2, 4)
fig, axs = plt.subplots(row, col, figsize=(5*col, 4*row), squeeze=False)

(Gc,iso,proj,preFactor)=('id','+','P0',1)
# (Gc,iso,proj,preFactor)=('gt','+','P0',1)
# (Gc,iso,proj,preFactor)=('g5gz','-','Pzt',-1j)
# (Gc,iso,proj,preFactor)=('g5gt','-','Pzt',1)
# (Gc,iso,proj,preFactor)=('g5','-','Pzt',1)
# (Gc,iso,proj,preFactor)=('g5','-','Pzt',1j)



tRef=1

colorDic = {0:'r',1:'g',2:'b',3:'purple'}
for i_pc in range(4):
    pc=[(0,0,0),(0,0,1),(0,0,-1),(1,0,0)][i_pc]
    dat=getDat(Gc,iso,proj,pc,2)

    axs[0,i_pc].set_title('pc='+str(pc))
    for i in range(2):
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            t=np.array([np.real(preFactor*t[ind_tf+2][:,0,i]) for ind_tf in range(len(tfList))],dtype=object)
            return t
        
        (mean,cov,err)=yu.jackknife(dat,tFunc)

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[0,i_pc].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt='s',color=colorDic[i],elinewidth=2,capsize=6,mfc='white')
        axs[0,i_pc].set_xticks(range(-6,6,2))
        # axs[0,i_pc].set_ylim(axs[0,0].get_ylim())

    for N_op in range(1,2+1):
        dat=getDat(Gc,iso,proj,pc,N_op)
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
            (eValsb,eVecsb)=yu.GEVP(t[0],tRef)
            def getRes(ind_tf):
                tf=tfList[ind_tf]
                eVeca=eVecsa[tf,:,0]
                eVecb=eVecsb[tf,:,0]
                denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
                np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
                nom=np.conj(eVeca).T@t[2+ind_tf]@eVecb
                return nom/denom
            t=np.array([np.real(preFactor*
                getRes(ind_tf)
                ) for ind_tf in range(len(tfList))],dtype=object)
            return t

        (mean,cov,err)=yu.jackknife(dat,tFunc)
        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            axs[1,i_pc].errorbar(np.arange(1 - tf//2,tf//2),tMean,tErr,\
                fmt='s',color=colorDic[N_op-1],elinewidth=2,capsize=6,mfc='white')
        axs[1,i_pc].set_xticks(range(-6,6,2))
        # axs[1,i_pc].set_ylim(axs[1,0].get_ylim())
        # axs[1,i_pc].set_yticks(axs[1,0].get_yticks())

In [ ]:
tfList=[10,12]

In [ ]:
def run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor,title=None,ylim1=None,ylim2=None,axs=None):
    tRef=4; tfAvgMin=tRef+1; tfAvgMax=15
    preFactorStr= 'real' if preFactor==1 else 'imag' if preFactor==-1j else preFactor

    if axs is None:
        fig, axs = yu.getFigAxs(1,1,Lrow=4*2,Lcol=5*2)
        projgsLabel = 'unPol' if 'P0' in projgs else 'Pol' 
    

    for projg in projgs:
        x_shift=0 if len(projgs)==1 else {'P0':0,'Pz':0,'P01':-0.3,'P02':0.3}[projg]
        flip = -1 if 'Pz' in projgs and projg=='P02' else 1

        irow=0; icol=1
        def tFunc(dat):
            t=np.mean(dat,axis=0)
            (eValsa,eVecsa)=yu.GEVP(t[0],tRef)
            (eValsb,eVecsb)=yu.GEVP(t[0],tRef)
            def getRes(ind_tf):
                tf=tfList[ind_tf]
                eVeca=eVecsa[tf,:,0]
                eVecb=eVecsb[tf,:,0]
                denom=np.sqrt( np.real(np.conj(eVeca).T@t[0][tf]@eVeca) * np.real(np.conj(eVecb).T@t[1][tf]@eVecb) *\
                np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca)[::-1]/np.real(np.conj(eVeca).T@t[0][1:tf]@eVeca) *\
                np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)/np.real(np.conj(eVecb).T@t[1][1:tf]@eVecb)[::-1] )
                nom=np.conj(eVeca).T@t[2+ind_tf]@eVecb
                return nom/denom
            t=np.array([np.real(preFactor*
                getRes(ind_tf)
                ) for ind_tf in range(len(tfList))],dtype=object)
            return t

        dat=getDat(Gc,iso,proj,(0,0,0),1)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        print('Red: without GEVP improvement')

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print('source sink separation: ',tf)
            for t_t in range(len(np.arange(1 - tf//2,tf//2))):
                print(np.arange(1 - tf//2,tf//2)[t_t],tMean[t_t],tErr[t_t])
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='r',elinewidth=2,capsize=6,mfc='white')

        axs[irow,0].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim1)

        icol=2
        dat=getDat(Gc,iso,proj,(0,0,0),2)
        (mean,cov,err)=yu.jackknife(dat,tFunc)
        
        colorDic={(0,0):'r',(0,1):'g',(0,2):'b',(0,3):'purple',(0,4):'orange'}
        fmtDic={10:'s',12:'d',14:'o'}

        print('Green: with GEVP improvement')

        for i_tf in range(len(tfList)):
            tf=tfList[i_tf]
            tMean=mean[i_tf];tErr=err[i_tf]
            print('source sink separation: ',tf)
            for t_t in range(len(np.arange(1 - tf//2,tf//2))):
                print(np.arange(1 - tf//2,tf//2)[t_t],tMean[t_t],tErr[t_t])
            # axs[irow,icol].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
            #     fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
            axs[irow,0].errorbar(np.arange(1 - tf//2,tf//2)+x_shift,tMean,tErr,\
                fmt=fmtDic[tf],color='g',elinewidth=2,capsize=6,mfc='white')
        # axs[irow,icol].set_xticks(range(-6,6,1))
        # axs[irow,icol].set_ylim(ylim2)

# pt1=(0,0,0); pt2=(0,0,1)
# proj1=(pt1,'a'); proj12=(pt2,'a')
# projg=(pt1,('N1\\pi1','a')); projg2=(pt2,('N1\\pi0','a')); projg3=(pt2,('N0\\pi1','a'))
# op1=('p',proj1); op12=('p',proj12)
# op2=('n&pi+',projg); op22=('n&pi+',projg2); op23=('n&pi+',projg3)
# op3=('p&pi0',projg); op32=('p&pi0',projg2); op33=('p&pi0',projg3)

pt0=(0,0,0); pt1=(0,0,1)
op01=('p',(pt0,'a')); op02=('12',(pt0,('N1\\pi1','a')))

(Gc,iso,projgs,preFactor)=('id','+',['P0'],1)

opas=[op01]; opbs=[op01,op02]
run3pt2ptGEVP(opas,opbs,Gc,iso,projgs,preFactor)